In [ ]:
from dask.distributed import Client,LocalCluster
import xarray as xr
import dask
import glob
import geopandas as gpd
import rioxarray as rio
import shapely
import os
os.environ['GDAL_DATA'] = os.environ['CONDA_PREFIX'] + r'/Library/share/gdal'
os.environ['PROJ_LIB'] = os.environ['CONDA_PREFIX'] + r'/Library/share'

In [ ]:
# data_dir=r'C://Users/kerrie/Documents/02_LocalData/nclimgrid_daily/'
# shpfile=r'C://Users/kerrie/Documents/02_LocalData/boundaries/study_area_bounding_box.shp'
data_dir=r'E://data/nclimgrid_daily/'
shpfile=r'E://data/boundaries/study_area_bounding_box.shp'

In [ ]:
# this may require adjustment on other computers

# kerrie laptop client (machine has 32GB RAM and 20 threads)
# nworkers=20  # set equal to number of threads you have

# kerrie desktop client (machine has 64GB RAM and 16 threads)
nworkers=16  # set equal to number of threads you have

cluster=LocalCluster(n_workers=nworkers,threads_per_worker=1)
client=Client(cluster) 
client

## Data Cleaning

1) drop any variables we don't need
2) dim names are time, lat, lon
3) time is correct datetimes
4) lat is ascending
5) lon is ascending -180 to 180
6) variable names tmax,tmin,prcp
7) units of C for temperature
8) units of mm/day for precip
9) spatial subset
11) round all values to 2 decimal places
12) save data with smallest possible precision
13) write cleaned files, 1 file per variable


In [ ]:
files=sorted(glob.glob(data_dir+'orig/ncdd*-grd-scaled.nc'))
len(files), files[0:3]

In [ ]:
# lazy dataset, all 800+ files
ds=xr.open_mfdataset(files,chunks='auto',lock=False)
ds

In [ ]:
# 1) drop data we don't need"
ds = ds.drop_vars('tavg')

steps 2-7 aren't necessary, data already looks good wrt to these items

In [ ]:
# step 8 millimeter --> mm/day
ds.prcp.attrs['units']='mm/day'

In [ ]:
# step 9 spatial subset
# clip data to a bounding box

# get clip object
box=gpd.read_file(shpfile)

# assign crs to netcdf data
ds.rio.write_crs("epsg:4326",inplace=True)
ds_clip=ds.rio.clip(box.geometry.apply(shapely.geometry.mapping),box.crs,drop=True,invert=False)
ds_clip

step 10 & 11, round and reduce precision

In [ ]:
# choosing a single time to test data precision, looking for values on order of at least 100
testtime='2000-06-4'
ds_clip.prcp.sel(time=testtime).plot()

In [ ]:
# float16 is probably not enough precision, let's check on a subset of the data

print('loading float16')
prcp_16 = ds_clip.prcp.sel(time=testtime).astype('float16').load()
print('loading float32')
prcp_32 = ds_clip.prcp.sel(time=testtime).astype('float32').load()
print('loading float64')
prcp_64 = ds_clip.prcp.sel(time=testtime).load()

prcp_16.max().item(),prcp_32.max().item(),prcp_64.max().item()

so we can change the data type from float64 to float32 but not go any smaller

In [ ]:
# steps 10 & 11
ds = ds_clip.round(decimals=2)
ds = ds_clip.astype('float32')
ds

# write files

In [ ]:
%%time
filename='prcp_nClimGridDaily_1951-2024_USsouth.nc'
print('writing',filename)
ds.prcp.to_netcdf(data_dir+filename)

In [ ]:
%%time
filename='tmax_nClimGridDaily_1951-2024_USsouth.nc'
print('writing',filename)
ds.tmax.to_netcdf(data_dir+filename)

In [ ]:
%%time
filename='tmin_nClimGridDaily_1951-2024_USsouth.nc'
print('writing',filename)
ds.tmin.to_netcdf(data_dir+filename)

In [ ]:
test=xr.open_mfdataset(data_dir+filename)
test

In [ ]:
client.shutdown()